In [1]:
%load_ext autoreload
%autoreload
from simba import transfer_function_to_graph, tf2rss, adiabatically_eliminate
from sympy import symbols, simplify, Matrix, sqrt, conjugate, lambdify

In [2]:
from simba.utils import construct_permutation_matrix

In [3]:
construct_permutation_matrix(6)

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 1]])

Tuned cavity

In [4]:
s = symbols('s')
gamma_f = symbols('gamma_f', real=True, positive=True)
tf = (s + gamma_f) / (s - gamma_f)
split_network = tf2rss(tf).to_slh().split()
gamma, = split_network.aux_coupling_constants

0


In [5]:
split_network.state_vector

Matrix([
[              a],
[   conjugate(a)],
[             a'],
[  conjugate(a')],
[            ain],
[ conjugate(ain)],
[           aout],
[conjugate(aout)]])

In [6]:
split_network.dynamical_matrix.eqns

Matrix([
[                                                                        -sqrt(gamma)*sqrt(gamma_f)*a'/s],
[                                                             -sqrt(gamma)*sqrt(gamma_f)*conjugate(a')/s],
[                                 sqrt(gamma)*sqrt(gamma_f)*a/s - sqrt(2)*sqrt(gamma)*ain/s + gamma*a'/s],
[sqrt(gamma)*sqrt(gamma_f)*conjugate(a)/s - sqrt(2)*sqrt(gamma)*conjugate(ain)/s + gamma*conjugate(a')/s],
[                                                                                                      0],
[                                                                                                      0],
[                                                                          -sqrt(2)*sqrt(gamma)*a' + ain],
[                                                    -sqrt(2)*sqrt(gamma)*conjugate(a') + conjugate(ain)]])

In [7]:
tf = split_network.tfm.open_loop('ain', 'aout')
tf

(gamma*gamma_f + gamma*s + s**2)/(gamma*gamma_f - gamma*s + s**2)

In [8]:
adiabatically_eliminate(tf, gamma).simplify()

(gamma_f + s)/(gamma_f - s)

In [9]:
tf = split_network.tfm.open_loop('a', 'aout').simplify()
tf

sqrt(2)*gamma*sqrt(gamma_f)/(gamma - s)

In [10]:
adiabatically_eliminate(tf, gamma).simplify()

sqrt(2)*sqrt(gamma_f)

In [11]:
tf = split_network.tfm.open_loop('ain', 'a').simplify()
tf

sqrt(2)*gamma*sqrt(gamma_f)/(gamma*gamma_f - gamma*s + s**2)

In [12]:
adiabatically_eliminate(tf, gamma).simplify()

sqrt(2)*sqrt(gamma_f)/(gamma_f - s)

In [13]:
split_network.interaction_hamiltonian.h

Matrix([
[0, 0, I*sqrt(gamma)*sqrt(gamma_f),                            0],
[0, 0,                           0, -I*sqrt(gamma)*sqrt(gamma_f)],
[0, 0,                           0,                            0],
[0, 0,                           0,                            0]])

First looking at passive realisation of coupled cavity setup with coupling constant $g = 0$

In [14]:
s = symbols('s')
gamma_f, omega_s = symbols('gamma_f omega_s', real=True, positive=True)
tf = (s**2 + s * gamma_f + omega_s**2) / (s**2 - s * gamma_f + omega_s**2)

transfer_function_to_graph(tf, 'passive_coupled_cavity.png', layout='dot')

0
wrote passive_coupled_cavity.png


![](passive_coupled_cavity.png)

In [15]:
split_network = tf2rss(tf).to_slh().split()

0


In [16]:
h_int = split_network.interaction_hamiltonian
h_int.expr.simplify()

I*(-sqrt(gamma_1)*sqrt(gamma_f)*a_1*conjugate(a'_1) + sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1)*a'_1 + omega_s*a_1*conjugate(a_2) - omega_s*conjugate(a_1)*a_2)

In [17]:
split_network.interaction_hamiltonian.h

Matrix([
[0, 0, -I*omega_s,         0, I*sqrt(gamma_1)*sqrt(gamma_f),                              0, 0, 0],
[0, 0,          0, I*omega_s,                             0, -I*sqrt(gamma_1)*sqrt(gamma_f), 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0]])

In [18]:
h_int.states

Matrix([
[            a_1],
[ conjugate(a_1)],
[            a_2],
[ conjugate(a_2)],
[           a'_1],
[conjugate(a'_1)],
[           a'_2],
[conjugate(a'_2)]])

In [19]:
simplify(h_int.dynamical_matrix)

Matrix([
[                           0,                            0, -omega_s,        0, sqrt(gamma_1)*sqrt(gamma_f),                           0, 0, 0],
[                           0,                            0,        0, -omega_s,                           0, sqrt(gamma_1)*sqrt(gamma_f), 0, 0],
[                     omega_s,                            0,        0,        0,                           0,                           0, 0, 0],
[                           0,                      omega_s,        0,        0,                           0,                           0, 0, 0],
[-sqrt(gamma_1)*sqrt(gamma_f),                            0,        0,        0,                           0,                           0, 0, 0],
[                           0, -sqrt(gamma_1)*sqrt(gamma_f),        0,        0,                           0,                           0, 0, 0],
[                           0,                            0,        0,        0,                           0,      

Looking at adiabatic elimination of $a_1'$

$\dot{a}_1' = -\gamma_1 a_1' - \sqrt{\gamma_1 \gamma_f} a_1 + \sqrt{2 \gamma_1} a_\text{in}$

adiabatic elimination: $\dot{a}_1' = 0$

$a_1' = \sqrt{\frac{\gamma_f}{\gamma_1}} a_1 - \sqrt{\frac{2}{\gamma_1}} a_\text{in}$

$H_\text{int} = i \sqrt{2\gamma_f}(a_\text{in}^\dagger a_1 - a_\text{in} a_1^\dagger)$

In [22]:
split_network.dynamical_matrix.states.states

Matrix([
[              a_1],
[   conjugate(a_1)],
[              a_2],
[   conjugate(a_2)],
[             a'_1],
[  conjugate(a'_1)],
[             a'_2],
[  conjugate(a'_2)],
[            ain_1],
[ conjugate(ain_1)],
[           aout_1],
[conjugate(aout_1)],
[            ain_2],
[ conjugate(ain_2)],
[           aout_2],
[conjugate(aout_2)]])

In [23]:
split_network.dynamical_matrix.eqns

Matrix([
[                                                                -sqrt(gamma_1)*sqrt(gamma_f)*a'_1/s + omega_s*a_2/s],
[                                          -sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a'_1)/s + omega_s*conjugate(a_2)/s],
[                                                                                                     -omega_s*a_1/s],
[                                                                                          -omega_s*conjugate(a_1)/s],
[                                 sqrt(gamma_1)*sqrt(gamma_f)*a_1/s - sqrt(2)*sqrt(gamma_1)*ain_1/s + gamma_1*a'_1/s],
[sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1)/s - sqrt(2)*sqrt(gamma_1)*conjugate(ain_1)/s + gamma_1*conjugate(a'_1)/s],
[                                                                                                                  0],
[                                                                                                                  0],
[                                      

In [24]:
# Calculating the input-output transfer function
tfm = split_network.tfm

tf = tfm.open_loop('ain_1', 'aout_1').simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

(gamma_f*s + omega_s**2 + s**2)/(gamma_f*s - omega_s**2 - s**2)

In [25]:
tf = tfm.open_loop('a_1', 'aout_1').simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

sqrt(2)*sqrt(gamma_f)

Now looking at the active realisation ($g \neq 0$)

In [26]:
# parameterise with lambda = g**2 - omega_s**2 > 0
lmbda = symbols('lambda', real=True, positive=True)
tf = (s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)

transfer_function_to_graph(tf, 'active_coupled_cavity.pdf', layout='dot')

0
wrote active_coupled_cavity.pdf


![](active_coupled_cavity.png)

In [27]:
split_network = tf2rss(tf).to_slh().split()
h_int = split_network.interaction_hamiltonian
h_int.expr.simplify()

0


I*(-sqrt(gamma_1)*sqrt(gamma_f)*a_1*conjugate(a'_1) + sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1)*a'_1 - sqrt(lambda)*a_1*a_2 + sqrt(lambda)*conjugate(a_1)*conjugate(a_2))

In [28]:
simplify(h_int.dynamical_matrix)

Matrix([
[                           0,                            0,            0, sqrt(lambda), sqrt(gamma_1)*sqrt(gamma_f),                           0, 0, 0],
[                           0,                            0, sqrt(lambda),            0,                           0, sqrt(gamma_1)*sqrt(gamma_f), 0, 0],
[                           0,                 sqrt(lambda),            0,            0,                           0,                           0, 0, 0],
[                sqrt(lambda),                            0,            0,            0,                           0,                           0, 0, 0],
[-sqrt(gamma_1)*sqrt(gamma_f),                            0,            0,            0,                           0,                           0, 0, 0],
[                           0, -sqrt(gamma_1)*sqrt(gamma_f),            0,            0,                           0,                           0, 0, 0],
[                           0,                            0,       

In [29]:
split_network.frequency_domain_eqns

AttributeError: 'SplitNetwork' object has no attribute 'frequency_domain_eqns'

In [ ]:
# Calculating the input-output transfer function
tfm = split_network.tfm

tf = tfm.open_loop('ain_1', 'aout_1').simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

In [ ]:
(s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)

Differs by phase shift of $\pi$

Now let's look at the transfer function from $a_1$ to $aout_1$, expect it to be frequency independent

In [ ]:
tf = tfm.open_loop('a_1', 'aout_1').simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)